In [4]:
!python -m pip install -r requirements.txt

  Obtaining dependency information for facebook-business from https://files.pythonhosted.org/packages/a7/c6/eefb3abdbb2b9ec6e433129ea3f3a57cb62fe9cbbd9e5a7c2acb99bbda2a/facebook_business-23.0.1-py3-none-any.whl.metadata
  Obtaining dependency information for curlify>=2.1.0 from https://files.pythonhosted.org/packages/9e/f8/912ebddbff8ea603d4c90fa31557096f927b17efd30a166ce7ac1242910a/curlify-3.0.0-py3-none-any.whl.metadata
  Obtaining dependency information for pycountry>=19.8.18 from https://files.pythonhosted.org/packages/b1/ec/1fb891d8a2660716aadb2143235481d15ed1cbfe3ad669194690b0604492/pycountry-24.6.1-py3-none-any.whl.metadata
  Obtaining dependency information for aiohttp from https://files.pythonhosted.org/packages/21/0c/74c895688db09a2852056abf32d128991ec2fb41e5f57a1fe0928e15151c/aiohttp-3.10.11-cp38-cp38-win_amd64.whl.metadata
  Obtaining dependency information for aiohappyeyeballs>=2.3.0 from https://files.pythonhosted.org/packages/b9/74/fbb6559de3607b3300b9be3cc64e97548d55678

In [2]:
import pandas as pd
import streamlit as st
import json
import io
import openai
import tabulate
import os
from datetime import date
from dotenv import load_dotenv, find_dotenv
from  meta_adds_connect import insights_meta as im

In [4]:
import os
from facebook_business.api import FacebookAdsApi
from facebook_business.adobjects.adaccount import AdAccount
_ = load_dotenv(find_dotenv())

APP_ID = os.getenv("META_APP_ID")
APP_SECRET = os.getenv("META_APP_SECRET")
ACCESS_TOKEN = os.getenv("META_ACCESS_TOKEN")
AD_ACCOUNT_ID = os.getenv("META_AD_ACCOUNT_ID")

FacebookAdsApi.init(app_id=APP_ID, app_secret=APP_SECRET, access_token=ACCESS_TOKEN)

account = AdAccount(AD_ACCOUNT_ID)

fields = [
    "date_start",           # Day
    "campaign_name",
    "adset_name",
    "ad_name",
    "spend",                # Amount Spent
    "impressions",
    "reach",
    "inline_link_clicks"   # Link Clicks (direto)
#    "actions"               # para Adds to Cart e Purchases
]

params = {
    "level": "ad",  # precisamos do nome do anúncio
    "time_range": {"since": "2025-08-01", "until": "2025-09-31"},
    "time_increment": 1,    # linha por dia (Day)
    "limit": 500
}

cursor = account.get_insights(fields=fields, params=params)
rows = [r.export_all_data() for r in cursor]

while cursor.load_next_page():
    rows.extend([r.export_all_data() for r in cursor])

In [5]:
start_str = '2025-07-01'
end_str = '2025-08-30' 
meta = im(APP_ID, APP_SECRET, ACCESS_TOKEN, AD_ACCOUNT_ID)
fields = ["date_start","campaign_name","adset_name","ad_name","spend","impressions","reach","inline_link_clicks"]
dados = meta.get_insights(fields = fields, since = f'{start_str}', until=f'{end_str}',time_increment = 'all_days')
print(dados)


[{'date_start': '2025-07-01', 'campaign_name': '[DSV] [BCP] [VENDAS] [AUTO] [Q] - Vendas BCP Quentes', 'adset_name': '00. [20-58] [IG/FB] Envolvimento 364D + Seguidores', 'ad_name': 'AD17 (LP02)', 'spend': '392.94', 'impressions': '28570', 'reach': '12626', 'inline_link_clicks': '525', 'date_stop': '2025-08-30'}, {'date_start': '2025-07-01', 'campaign_name': '[DSV] [BCP] [VENDAS] [AUTO] [Q] - Vendas BCP Quentes', 'adset_name': '00. [20-58] [IG/FB] Envolvimento 364D + Seguidores', 'ad_name': 'AD37 (LP02)', 'spend': '35.5', 'impressions': '2501', 'reach': '1218', 'inline_link_clicks': '41', 'date_stop': '2025-08-30'}, {'date_start': '2025-07-01', 'campaign_name': '[DSV] [BCP] [VENDAS] [AUTO] [Q] - Vendas BCP Quentes', 'adset_name': '00. [20-58] [IG/FB] Envolvimento 364D + Seguidores', 'ad_name': 'AD04', 'spend': '164.15', 'impressions': '9105', 'reach': '4728', 'inline_link_clicks': '208', 'date_stop': '2025-08-30'}, {'date_start': '2025-07-01', 'campaign_name': '[DSV] [BCP] [VENDAS] [AU

In [ ]:
client = openai.OpenAI()
resp = client.responses.create(
    model="gpt-4.1-mini",  # modelo de texto compatível com tools
    input=[
        {
            "role": "developer",
            "content": f'''
                Você receberá no user input uma lista de dicionários contendo resultados de campanhas do cliente. 
                Atue como um **analista de marketing de dados sênior** e produza um **report estruturado em Markdown** no seguinte formato: 
                📊 Reporte de Performance – Campanhas 
                    1. Resumo Executivo  
                        - 🔥 Destaque dos ads com os **melhores desempenhos** 
                            - (iformar sobre métricas como CTR, CPC, CPM, cliques) 
                        - ❌ Destaque os **piores desempenhos** 
                    2. Recomendações 
                        - 💰 Sugira **alocação de budget** (onde investir mais e onde reduzir no formato bullet points) 
                        - 🧪 Sugira **testes A/B práticos** (criativos, públicos, LPs, etc.) 
                Regras: 
                    - Use métricas por criativo/adset/campanha quando fizer sentido. 
                    - Retorne **sempre formatado em Markdown**, com ícones e divisórias para ficar visual. 
                    - O texto deve ter no máximo **2–3 blocos curtos por seção**.
            '''
        },
        {
            "role": "user",
            "content": [
                # dados como JSON em texto:
                {"type": "input_text", "text": json.dumps(dados, ensure_ascii=False)}
            ]
        }
    ],
)

In [7]:
resp

Response(id='resp_68be20e50dc08194b1268be93e8573090d4cd026c1a86b43', created_at=1757290725.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4.1-mini-2025-04-14', object='response', output=[ResponseOutputMessage(id='msg_68be20e580688194bb82b39c36106fc20d4cd026c1a86b43', content=[ResponseOutputText(annotations=[], text='```markdown\n# 📊 Reporte de Performance – Campanhas\n\n---\n\n## 1. Resumo Executivo\n\n### 🔥 Melhores Desempenhos  \n- **AD17 (LP02) - Vendas BCP Quentes**  \n  - Spend: R$392,94 | Impressões: 28.570 | Cliques: 525  \n  - CTR: 1,84% | CPC: R$0,75 | CPM: R$13,75  \n  - Maior volume de cliques e boa eficiência no CPC.  \n- **CTA BCP - 3 Erros mais comuns - Vendas BCP Quentes**  \n  - Spend: R$175,13 | Impressões: 12.838 | Cliques: 255  \n  - CTR: 1,99% | CPC: R$0,69 | CPM: R$13,64  \n  - Excelente CTR e custo por clique competitivo.\n\n### ❌ Piores Desempenhos  \n- **AD37 (LP02) - Vendas BCP Frios**  \n  - Spend: R$32,32 | Impressões: 1.66